# Netflix Stock Price Prediction ML Project

#### Mohamed Youssef - 211001821


## Index

* [Including Necessary Libraries](#libs)
<br>
* [Data Preprocessing & Cleaning](#dataset)
<br>
* [Data Visualization](#vis)
<br>
* [Random Forest Model](#m1)
<br>
* [Polynomial Regression Model](#m2)
<br>
* [Ada Boost Model](#m3)
<br>
* [Model Comparison](#compare)


## Including Necessary Libraries <a id='libs'></a>


In [1]:
# Python built-in libraries
import datetime as dt
import time
import warnings

# Libraries for preprocessing and visualizing the data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# ML models evaluation metrics
from sklearn import metrics

# Sklearn ML models classes
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import AdaBoostRegressor